In [ ]:
import pm4py

In [ ]:
path = "log.jsonocel"
ocel = pm4py.read_ocel(path)
ocdfg = pm4py.discover_ocdfg(ocel)
# views the model with the performance annotation
pm4py.view_ocdfg(ocdfg, format="png", annotation="performance", performance_aggregation="median")


In [ ]:
path = "tmp5hghlgef.xmlocel"
ocel = pm4py.read_ocel(path)
ocel

In [ ]:
model = pm4py.discover_oc_petri_net(ocel)
pm4py.view_ocpn(model, format="png")

In [ ]:
path = "p2p-normal.jsonocel"
ocel = pm4py.read_ocel(path)
ocdfg = pm4py.discover_ocdfg(ocel)
# views the model with the performance annotation
pm4py.view_ocdfg(ocdfg, format="png", annotation="performance", performance_aggregation="median")


In [ ]:
model = pm4py.discover_oc_petri_net(ocel)
pm4py.view_ocpn(model, format="png")